In [30]:
import requests
from bs4 import BeautifulSoup
import pathlib
from datetime import datetime
import os
import time
import sys
sys.path.append('.')

from utils import *
from compress_utils import compressed_cpickle, decompressed_cpickle

In [74]:
def get_topic(soup, href=True):
    
    h3_soup = soup.find_all('h3', class_="gs_rt")
    gs_rs_soup = soup.find_all('div', class_="gs_rs")
    gs_a_group = soup.find_all('div', class_='gs_a')
    result = {'topic':[], 'abstract':[], "year":[]}
    
    valid_website = ['openaccess.thecvf.com', 'proceedings.neurips.cc', 'link.springer.com', 'dl.acm.org',
         'arxiv.org', 'www.pnas.org', 'www.usenix.org', 'www.sciencedirect.com', 'www.nature.com',
          'proceedings.mlr.press']
    except_lis = ['ieeexplore.ieee.org']
    valid_website = [i for i in valid_website if not i in except_lis]
    
    text_rule = lambda text:text.replace('.','_')
    extract_func_dic = {}
    for website in valid_website:
        name_ = 'abs_from_'+text_rule(website)
        extract_func_dic[website] = eval(name_)
    

    if href:
        result['href'] = []
        
                
    # get topic and abstract and href
    for topic_res, abstract_res, attr_res in zip(h3_soup, gs_rs_soup, gs_a_group):
        # get hef:
        if href:
            a = str(topic_res).split(' ')
            href_ = [e[6:-1] for e in a if e.__contains__('href')][0]
    #             print(href_)
            result['href'].append(href_)
        
        # get year
        for i in [str(year) for year in range(2015,2022)]:
            if attr_res.get_text().__contains__(i):
                result['year'].append(i)
#                 print(i)
        
        # get topic
        topic_text = topic_res.get_text()
        b = '[HTML][HTML] '
        topic_text = topic_text.replace(b, '')
        result['topic'].append(topic_text)
        
        
        # get abstract
        website = href_.split('/')[2] 
        if [i for i in valid_website if i.__contains__(website)]:
            abstract = extract_func_dic[website](href_)
            result['abstract'].append(abstract)
        else:
            # google scholar default short abstract
            result['abstract'].append(abstract_res.get_text())
    
    return result

In [93]:
# test code
# google_url = "https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5"
# r = requests.get(google_url)
# soup = BeautifulSoup(r.text, 'html.parser')
# res = get_topic(soup)
# print(res)

# for contents in zip(*res.values()):
#     print(contents)
#     for i in range(len(contents)):
#         print(list(res.keys())[i], contents[i])

page
- [ok]. 0~30
- [ok]. 30~60
- [ok]. 60~90
- [ok]. 90~120

In [158]:
google_url = "https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5"
end = 980
dic = {}
for i in range(end//10+1)[90:120]:
    a = google_url + f'&start={i*10}'
    print(a)
    r = requests.get(a)
    
    soup = BeautifulSoup(r.text, 'html.parser')
    res = get_topic(soup)
#     print(res)
#     lis.append()
    dic['page_'+str(i)] = res
    time.sleep(5)
    

https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=900
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=910
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=920
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=930
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=940
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=950
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=960
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=970
https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=980


In [159]:
dic.keys()

dict_keys(['page_90', 'page_91', 'page_92', 'page_93', 'page_94', 'page_95', 'page_96', 'page_97', 'page_98'])

In [160]:
from datetime import datetime
import pathlib

path = './Data'
pathlib.Path(f'{path}').mkdir(parents=True, exist_ok=True)  

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m-%H%M")
# print(dt_string)
compressed_cpickle(f'./Data/scholar_analysis_FL_{dt_string}', dic)


In [161]:
import os
import time

pbz_paths = ['/'.join([path,i]) for i in os.listdir(path) if i.__contains__('.pbz2')]
print(pbz_paths)
newest_file_path = max(pbz_paths, key=os.path.getctime)
newest_file_path

['./Data/scholar_analysis_FL_1401-1217.pbz2', './Data/scholar_analysis_FL_1401-1545.pbz2', './Data/scholar_analysis_FL_1401-1633.pbz2', './Data/scholar_analysis_FL_1401-1643.pbz2']


'./Data/scholar_analysis_FL_1401-1643.pbz2'

In [162]:
newest_file_path

'./Data/scholar_analysis_FL_1401-1643.pbz2'

In [163]:
res.keys()

dict_keys(['topic', 'abstract', 'year', 'href'])

In [164]:
# path = './Data/scholar_analysis_FL_1401-1217.pbz2'
path = newest_file_path
res = decompressed_cpickle(path)
page_res = res[list(res.keys())[0]]

for contents in zip(*page_res.values()):
#     print(contents)
    for i in range(len(contents)):
        print("<<",list(page_res.keys())[i],">>", contents[i])
    print('')

<< topic >> Privacy-preserving spatiotemporal scenario generation of renewable energies: A federated deep generative learning approach
<< abstract >> … Besides, an experiment with different federated learning settings is designed and conducted 
to verify the robustness of our method. … an emerging and powerful distributed machine 
learning technology— federated learning. Federated learning is initially proposed by Google …
<< year >> 2021
<< href >> https://ieeexplore.ieee.org/abstract/document/9492026/

<< topic >> A Preliminary Scoping Study of Federated Learning for the Internet of Medical Things
<< abstract >> … This paper presents a scoping review of federated learning for the Internet of Medical Things 
(… machine learning and avoiding the need to centrally compile patient data, federated learning 
… maps the literature relevant to Federated Learning and IoMT and identifies key concepts. …
<< year >> 2021
<< href >> https://ebooks.iospress.nl/doi/10.3233/SHTI210216

<< topic >> I